In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
import shap

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [ ]:
df.head()

In [ ]:
df.columns

# --- DATA PREPROCESSING ---


In [ ]:
# Fix TotalCharges
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'].fillna(df['TotalCharges'].mean(), inplace=True)

In [ ]:
df_for_dashboard = df.copy()

In [ ]:
# Drop ID for training
df = df.drop('customerID', axis=1)

In [ ]:
df_encoded = pd.get_dummies(df, drop_first=True)

In [ ]:
X = df_encoded.drop('Churn_Yes', axis=1)
y = df_encoded['Churn_Yes']

In [ ]:
X_train, X_test, y_train, y_test, df_dashboard_train, df_dashboard_test = train_test_split(
    X, y, df_for_dashboard, test_size=0.2, random_state=42
)

In [ ]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model.fit(X_train_resampled, y_train_resampled)

In [ ]:
export_df = df_dashboard_test.copy()

In [ ]:
export_df['Predicted_Churn'] = model.predict(X_test)
export_df['Churn_Probability'] = model.predict_proba(X_test)[:, 1]

In [ ]:
export_df.to_csv('churn_data.csv', index=False)
print("Data exported with original Text Columns + Predictions!")

In [ ]:
from sklearn.metrics import accuracy_score

score = accuracy_score(y_test, model.predict(X_test))
print(f"My Model Accuracy is: {score:.2%}")